#Setup

In [ ]:
!nvidia-smi
!pip install einops
!pip install pyyaml==5.4.1
!pip install transformers
!pip install git+https://github.com/aVariengien/Easy-Transformer

Tue Aug 30 02:14:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
import tqdm.notebook as tqdm

import random
import time

from google.colab import drive
from pathlib import Path
import pickle
import os

import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "colab"
import plotly.graph_objects as go

from torch.utils.data import DataLoader

from functools import *
import pandas as pd
import gc
import collections
import copy

# import comet_ml
import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer



In [ ]:
from easy_transformer.utils import gelu_new, to_numpy, get_corner #helper functions
from easy_transformer.hook_points import HookedRootModule, HookPoint
from easy_transformer.EasyTransformer import EasyTransformer,TransformerBlock, MLP, Attention, LayerNormPre, PosEmbed, Unembed, Embed
from easy_transformer.EasyTransformer import AblationMetric, AblationConfig, EasyAblation


#Hook Points

A Garcon-style interface - the key thing is a HookPoint class. This is a layer to wrap any activation within the model in. The HookPoint acts as an identity function, but allows us to put PyTorch hooks in to edit and access the relevant activation. This allows us to take any model and insert in access points to all interesting activations by wrapping them in HookPoints

There is also a `HookedRootModule` class - this is a utility class that the root module should inherit from (root module = the model we run) - it has several utility functions for using hooks well. 

The default interface is the `run_with_hooks` function on the root module, which lets us run a forwards pass on the model, and pass on a list of hooks paired with layer names to run on that pass. 

The syntax for a hook is `function(activation, hook)` where `activation` is the activation the hook is wrapped around, and `hook` is the `HookPoint` class the function is attached to. If the function returns a new activation or edits the activation in-place, that replaces the old one, if it returns None then the activation remains as is.



## Hook Points Example

Here's a simple example of how to use the classes:

We define a basic network with two layers that each take a scalar input $x$, square it, and add a constant:
$x_0=x$, $x_1=x_0^2+3$, $x_2=x_1^2-4$.

We wrap the input, each layer's output, and the intermediate value of each layer (the square) in a hook point.

In [ ]:
from easy_transformer.hook_points import HookedRootModule, HookPoint

class SquareThenAdd(nn.Module):
    def __init__(self, offset):
        super().__init__()
        self.offset = nn.Parameter(torch.tensor(offset))
        self.hook_square = HookPoint()
    
    def forward(self, x):
        # The hook_square doesn't change the value, but lets us access it
        square = self.hook_square(x * x)
        return self.offset + square
    
class TwoLayerModel(HookedRootModule):
    def __init__(self):
        super().__init__()
        self.layer1 = SquareThenAdd(3.)
        self.layer2 = SquareThenAdd(-4.)
        self.hook_in = HookPoint()
        self.hook_mid = HookPoint()
        self.hook_out = HookPoint()

        # We need to call the setup function of HookedRootModule to build an 
        # internal dictionary of modules and hooks, and to give each hook a name
        super().setup()
    
    def forward(self, x):
        # We wrap the input and each layer's output in a hook - they leave the 
        # value unchanged (unless there's a hook added to explicitly change it), 
        # but allow us to access it.
        x_in = self.hook_in(x)
        x_mid = self.hook_mid(self.layer1(x_in))
        x_out = self.hook_out(self.layer2(x_mid))
        return x_out
model = TwoLayerModel()



We can add a cache, to save the activation at each hook point

(There's a custom `cache_all` function on the root module as a convenience, which will add hooks to cache every activation at a hook point - we could also manually add hooks with `run_with_hooks`)

In [ ]:
cache = {}
model.cache_all(cache)
print('Model output:', model(torch.tensor(5.)).item())
for key in cache:
    print(f"Value cached at hook {key}", cache[key].item())

Model output: 780.0
Value cached at hook hook_in 5.0
Value cached at hook layer1.hook_square 25.0
Value cached at hook hook_mid 28.0
Value cached at hook layer2.hook_square 784.0
Value cached at hook hook_out 780.0


We can also use hooks to intervene on activations - eg, we can set the intermediate value in layer 2 to zero to change the output to -5

In [ ]:
def set_to_zero_hook(tensor, hook):
    print(hook.name)
    return torch.tensor(0.)
print('Output after intervening on layer2.hook_scaled', 
      model.run_with_hooks(torch.tensor(5.),
                           fwd_hooks = [('layer2.hook_square', set_to_zero_hook)]).item())

layer2.hook_square
Output after intervening on layer2.hook_scaled -4.0


# Transformer models

We now define a stripped down transformer. There are helper functions to load in the weights of several families of open source LLMs - OpenAI's GPT-2, Facebook's OPT and Eleuther's GPT-Neo.

Note: OPT-350M is not supported - it applies the LayerNorms to the *outputs* of each layer, which means we cannot fold the weights and biases into other layers, and would require notably different architecture.

**TODO:** Add in GPT-J and GPT-NeoX functionality

The list of supported model names:
```
 ['gpt2', 
                     'gpt2-medium', 
                     'gpt2-large', 
                     'gpt2-xl', 
                     'facebook/opt-125m', 
                     'facebook/opt-1.3b', 
                     'facebook/opt-2.7b', 
                     'facebook/opt-6.7b', 
                     'facebook/opt-13b', 
                     'facebook/opt-30b', 
                     'facebook/opt-66b', 
                     'EleutherAI/gpt-neo-125M', 
                     'EleutherAI/gpt-neo-1.3B', 
                     'EleutherAI/gpt-neo-2.7B', 
                     'EleutherAI/gpt-j-6B', 
                     'EleutherAI/gpt-neox-20b']
                     ```

#Examples

##Setup

Load in GPT-2 small

In [ ]:
model_name = 'gpt2' #@param ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl', 'facebook/opt-125m', 'facebook/opt-1.3b', 'facebook/opt-2.7b', 'facebook/opt-6.7b', 'facebook/opt-13b', 'facebook/opt-30b', 'facebook/opt-66b', 'EleutherAI/gpt-neo-125M', 'EleutherAI/gpt-neo-1.3B', 'EleutherAI/gpt-neo-2.7B', 'EleutherAI/gpt-j-6B', 'EleutherAI/gpt-neox-20b']
model = EasyTransformer(model_name)
if torch.cuda.is_available():
    model.to('cuda')

Create some reference text to run the models on.

In [ ]:
prompt = 'Interpretability is great'
tokens = model.to_tokens(prompt)
prompt_2 = 'AI Alignment is great'
tokens_2 = model.to_tokens(prompt_2)
def show_tokens(tokens):
    # Prints the tokens as text, separated by |
    if type(tokens)==str:
        # If we input text, tokenize first
        tokens = model.to_tokens(tokens)
    text_tokens = [model.tokenizer.decode(t) for t in tokens.squeeze()]
    print('|'.join(text_tokens))
show_tokens(tokens)
show_tokens(tokens_2)

Inter|pret|ability| is| great
AI| Al|ignment| is| great


In [ ]:
model.reset_hooks()
original_logits = model(tokens)
print('Top corner of logits')
print(get_corner(original_logits, 4))

Top corner of logits
tensor([[[-5.0197, -4.0007, -6.4540, -6.6005],
         [-4.1477, -2.2966, -7.4325, -6.9754],
         [-2.7587,  1.3903, -4.3042, -6.5975],
         [-6.1653, -5.0678, -9.1324, -9.5672]]], device='cuda:0',
       grad_fn=<SliceBackward0>)


In [ ]:
print('Reference: Hyperparameters for the model')
for hyper_param in (model.cfg):
    print(hyper_param, model.cfg[hyper_param])

Reference: Hyperparameters for the model
d_model 768
d_head 64
n_heads 12
d_mlp 3072
n_layers 12
n_ctx 1024
eps 1e-05
d_vocab 50257
act_fn gelu_new
use_attn_scale True
use_local_attn False
model_name gpt2
model_type gpt2
use_attn_result False


##Using the model

The model can be given either text or tokens as an input (text is automatically converted to a `batch_size=1` batch of tokens)

In [ ]:
logits_tokens = model(tokens)
logits_text = model(prompt)

The model gives the same log_probs as the original Hugging Face model 

Though *not* the same logits, as we remove a constant offset from $W_U$

In [ ]:
original_model = AutoModelForCausalLM.from_pretrained(model_name)
easy_logits = model(tokens).cpu()
original_model_logits = original_model(tokens).logits

easy_log_probs = F.log_softmax(easy_logits, dim=-1)
original_model_log_probs = F.log_softmax(original_model_logits, dim=-1)

print('Fraction of log probs the same between easy model and original model:')
print(torch.isclose(original_model_log_probs, easy_log_probs).sum()/easy_log_probs.numel())
print('Fraction of logits the same between easy model and original model:')
print(torch.isclose(original_model_logits, easy_logits).sum()/easy_logits.numel())

Fraction of log probs the same between easy model and original model:
tensor(1.)
Fraction of logits the same between easy model and original model:
tensor(0.)


##Basic Examples

Print the shapes of all activations

**Note:** This cell is a good reference for creating hooks - it's extremely useful to know the shapes of different activations as accessible by each hook!

By convention, each activation is batch x position x ... (where the final dimension(s) is d_model, (head_index x d_head) or d_mlp). The one exception is hook_attn (attention patterns) which has shape batch x head_index x query_pos x key_pos

**Reference:**
`batch_size=4
n_ctx=50
d_head=64
d_model=768
d_mlp=3072
n_heads=12
n_layers=12`

In [ ]:
all_hooks_fn = lambda name: True
def print_shape(tensor, hook):
    print(f'Activation at hook {hook.name} has shape:')
    print(tensor.shape)
random_tokens = torch.randint(1000, 10000, (4, 50))
logits = model.run_with_hooks(random_tokens, fwd_hooks=[(all_hooks_fn, print_shape)])

Activation at hook hook_embed has shape:
torch.Size([4, 50, 768])
Activation at hook hook_pos_embed has shape:
torch.Size([50, 768])
Activation at hook blocks.0.hook_resid_pre has shape:
torch.Size([4, 50, 768])
Activation at hook blocks.0.ln1.hook_scale has shape:
torch.Size([4, 50, 1])
Activation at hook blocks.0.attn.hook_q has shape:
torch.Size([4, 50, 12, 64])
Activation at hook blocks.0.attn.hook_k has shape:
torch.Size([4, 50, 12, 64])
Activation at hook blocks.0.attn.hook_v has shape:
torch.Size([4, 50, 12, 64])
Activation at hook blocks.0.attn.hook_attn_scores has shape:
torch.Size([4, 12, 50, 50])
Activation at hook blocks.0.attn.hook_attn has shape:
torch.Size([4, 12, 50, 50])
Activation at hook blocks.0.attn.hook_z has shape:
torch.Size([4, 50, 12, 64])
Activation at hook blocks.0.hook_attn_out has shape:
torch.Size([4, 50, 768])
Activation at hook blocks.0.hook_resid_mid has shape:
torch.Size([4, 50, 768])
Activation at hook blocks.0.ln2.hook_scale has shape:
torch.Size([4

Print the top corner of all activations

**Note:** This is useful to do as a sanity check when debugging a model, to quickly and roughly compare the new activations to the original activations (without looking at the full enormous tensors)

In [ ]:
def print_corner(tensor, hook):
    print(hook.name)
    print(get_corner(tensor))
logits = model.run_with_hooks(tokens, fwd_hooks=[(all_hooks_fn, print_corner)])

hook_embed
tensor([[[ 0.1600, -0.1444],
         [-0.0406, -0.2098]]], device='cuda:0', grad_fn=<SliceBackward0>)
hook_pos_embed
tensor([[-0.0134, -0.1920],
        [ 0.0250, -0.0528]], device='cuda:0', grad_fn=<SliceBackward0>)
blocks.0.hook_resid_pre
tensor([[[ 0.1466, -0.3363],
         [-0.0156, -0.2626]]], device='cuda:0', grad_fn=<SliceBackward0>)
blocks.0.ln1.hook_scale
tensor([[[0.3703],
         [0.2421]]], device='cuda:0', grad_fn=<SliceBackward0>)
blocks.0.attn.hook_q
tensor([[[[-0.6830,  0.1875],
          [ 0.5510,  0.1701]],

         [[ 0.4226,  0.8636],
          [ 0.1361, -0.6476]]]], device='cuda:0', grad_fn=<SliceBackward0>)
blocks.0.attn.hook_k
tensor([[[[-1.1447,  2.1864],
          [ 1.4646,  0.4051]],

         [[-1.5435,  2.9672],
          [ 0.9403, -1.4134]]]], device='cuda:0', grad_fn=<SliceBackward0>)
blocks.0.attn.hook_v
tensor([[[[-0.0110,  0.0460],
          [ 0.4635,  0.0313]],

         [[ 0.1250, -0.3105],
          [ 0.3096,  0.2382]]]], device='cuda:

Cache all activations


In [ ]:
cache = {}
model.reset_hooks()
model.cache_all(cache)
logits = model(tokens)
for name in cache:
    print(name, cache[name].shape)
model.reset_hooks()

hook_embed torch.Size([1, 5, 768])
hook_pos_embed torch.Size([5, 768])
blocks.0.hook_resid_pre torch.Size([1, 5, 768])
blocks.0.ln1.hook_scale torch.Size([1, 5, 1])
blocks.0.attn.hook_q torch.Size([1, 5, 12, 64])
blocks.0.attn.hook_k torch.Size([1, 5, 12, 64])
blocks.0.attn.hook_v torch.Size([1, 5, 12, 64])
blocks.0.attn.hook_attn_scores torch.Size([1, 12, 5, 5])
blocks.0.attn.hook_attn torch.Size([1, 12, 5, 5])
blocks.0.attn.hook_z torch.Size([1, 5, 12, 64])
blocks.0.hook_attn_out torch.Size([1, 5, 768])
blocks.0.hook_resid_mid torch.Size([1, 5, 768])
blocks.0.ln2.hook_scale torch.Size([1, 5, 1])
blocks.0.mlp.hook_pre torch.Size([1, 5, 3072])
blocks.0.mlp.hook_post torch.Size([1, 5, 3072])
blocks.0.hook_mlp_out torch.Size([1, 5, 768])
blocks.0.hook_resid_post torch.Size([1, 5, 768])
blocks.1.hook_resid_pre torch.Size([1, 5, 768])
blocks.1.ln1.hook_scale torch.Size([1, 5, 1])
blocks.1.attn.hook_q torch.Size([1, 5, 12, 64])
blocks.1.attn.hook_k torch.Size([1, 5, 12, 64])
blocks.1.attn.h

To save GPU memory, we can cache activations to the CPU - note that this is much slower though, since it requires copying.

In [ ]:
random_tokens = torch.randint(1000, 10000, (1, 300))
cache = {}
model.reset_hooks()
model.cache_all(cache, device='cpu')
print('Run time when copying to the CPU')
%timeit logits = model(random_tokens)
model.reset_hooks()
model.cache_all(cache, device='cuda')
print('Run time when just caching on GPU')
%timeit logits = model(random_tokens)

Run time when copying to the CPU
173 ms ± 3.12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Run time when just caching on GPU
31.7 ms ± 480 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


##Editing Activations
**To change an activation, add a hook to that HookPoint which returns the new activation**

Pruning attention heads

In [ ]:
# Example - prune heads 0, 3 and 7 from layer 3 and heads 8 and 9 from layer 7
layer = 3
head_indices = torch.tensor([0, 3, 7])
layer_2 = 7
head_indices_2 = torch.tensor([8, 9])
def prune_fn_1(z, hook):
    # The shape of the z tensor is batch x pos x head_index x d_head
    z[:, :, head_indices, :] = 0.
    return z
def prune_fn_2(z, hook):
    # The shape of the z tensor is batch x pos x head_index x d_head
    z[:, :, head_indices_2, :] = 0.
    return z
logits = model.run_with_hooks(tokens, fwd_hooks=[(f'blocks.{layer}.attn.hook_z', prune_fn_1),
                                                       (f'blocks.{layer_2}.attn.hook_z', prune_fn_2)])

Restrict all attention heads to only attend to the current and previous token.

**Validation:** The logits for the first 2 positions are the same, the logits for pos 3 are different

In [ ]:
model.reset_hooks()
def filter_hook_attn(name):
    split_name = name.split('.')
    return (split_name[-1]=='hook_attn')
def restrict_attn(attn, hook):
    # Attn has shape batch x head_index x query_pos x key_pos
    n_ctx = attn.size(-2)
    key_pos = torch.arange(n_ctx)[None, :]
    query_pos = torch.arange(n_ctx)[:, None]
    mask = (key_pos>(query_pos-2)).cuda()
    ZERO = torch.tensor(0.)
    if torch.cuda.is_available():
        ZERO = ZERO.cuda()
    attn = torch.where(mask, attn, ZERO)
    return attn
logits = model.run_with_hooks(tokens, fwd_hooks=[(filter_hook_attn, restrict_attn)])
print('New logits')
print(get_corner(logits, 3))
print('Original logits')
print(get_corner(original_logits, 3))

New logits
tensor([[[-5.0197, -4.0007, -6.4540],
         [-4.1477, -2.2966, -7.4325],
         [-2.6209,  2.9651, -4.4607]]], device='cuda:0',
       grad_fn=<SliceBackward0>)
Original logits
tensor([[[-5.0197, -4.0007, -6.4540],
         [-4.1477, -2.2966, -7.4325],
         [-2.7587,  1.3903, -4.3042]]], device='cuda:0',
       grad_fn=<SliceBackward0>)


Freezing attention patterns - here we do two runs of the model. First on the original text, caching attn patterns, and secondly on the new text, loading the cached patterns


In [ ]:
attn_cache = {}
def cache_attn(attn, hook):
    attn_cache[hook.name]=attn

def freeze_attn(attn, hook):
    return attn_cache[hook.name]

logits = model.run_with_hooks(tokens, fwd_hooks=[(filter_hook_attn, cache_attn)])

logits_2 = model.run_with_hooks(tokens_2, fwd_hooks=[(filter_hook_attn, freeze_attn)])

##Using Hook Contexts

**Each hook point has a dictionary `hook.ctx` that can be used to store information between runs** - this is useful for keeping running totals, etc 

A running total of times a neuron activation was positive


In [ ]:
# We focus on neuron 20 in layer 7
model.reset_hooks()
animal_texts = ['The dog was green', 'The cat was blue', 'The squid was magenta', 'The blobfish was grey']
layer = 7
neuron_index = 20
def running_total_hook(neuron_acts, hook):
    if 'total' not in hook.ctx:
        hook.ctx['total']=0
    print('Neuron acts:', neuron_acts[0, :, neuron_index])
    hook.ctx['total']+=(neuron_acts[0, :, neuron_index]>0).sum().item()
    print('Running total:', hook.ctx['total'])

for animal_text in animal_texts:
    show_tokens(animal_text)
    model.run_with_hooks(animal_text, fwd_hooks=[(f'blocks.{layer}.mlp.hook_post', running_total_hook)])

The| dog| was| green
Neuron acts: tensor([-0.0099, -0.1396,  0.6828, -0.0826], device='cuda:0',
       grad_fn=<SelectBackward0>)
Running total: 1
The| cat| was| blue
Neuron acts: tensor([-0.0099, -0.1045,  0.6142, -0.0625], device='cuda:0',
       grad_fn=<SelectBackward0>)
Running total: 2
The| squid| was| mag|enta
Neuron acts: tensor([-0.0099,  0.7520,  0.7486, -0.0986, -0.0460], device='cuda:0',
       grad_fn=<SelectBackward0>)
Running total: 4
The| blob|fish| was| grey
Neuron acts: tensor([-0.0099,  0.0554,  0.4716,  0.7331, -0.0160], device='cuda:0',
       grad_fn=<SelectBackward0>)
Running total: 7


Finding the dataset example that most activates a given neuron


In [ ]:
# We focus on neuron 13 in layer 5
model.reset_hooks(clear_contexts=True)
animal_texts = ['The dog was green', 'The cat was blue', 'The squid was magenta', 'The blobfish was grey']
layer = 5
neuron_index = 13
def best_act_hook(neuron_acts, hook, text):
    if 'best' not in hook.ctx:
        hook.ctx['best']=-1e3
    print('Neuron acts:', neuron_acts[0, :, neuron_index])
    if hook.ctx['best']<neuron_acts[0, :, neuron_index].max():
        print(f'Updating best act from {hook.ctx["best"]} to {neuron_acts[0, :, neuron_index].max().item()}')
        hook.ctx['best'] = neuron_acts[0, :, neuron_index].max().item()
        hook.ctx['text'] = text

for animal_text in animal_texts:
    (show_tokens(animal_text))
    # Use partial to give the hook access to the relevant text
    model.run_with_hooks(animal_text, fwd_hooks=[(f'blocks.{layer}.mlp.hook_post', partial(best_act_hook, text=animal_text))])
print()
print('Maximally activating dataset example:', model.hook_dict[f'blocks.{layer}.mlp.hook_post'].ctx['text'])
model.reset_hooks(clear_contexts=True)

The| dog| was| green
Neuron acts: tensor([-0.0074, -0.1690,  0.0724,  0.0520], device='cuda:0',
       grad_fn=<SelectBackward0>)
Updating best act from -1000.0 to 0.07240132987499237
The| cat| was| blue
Neuron acts: tensor([-0.0074, -0.1681,  0.1947,  0.0884], device='cuda:0',
       grad_fn=<SelectBackward0>)
Updating best act from 0.07240132987499237 to 0.19472411274909973
The| squid| was| mag|enta
Neuron acts: tensor([-0.0074, -0.1546,  0.0558, -0.1591, -0.1391], device='cuda:0',
       grad_fn=<SelectBackward0>)
The| blob|fish| was| grey
Neuron acts: tensor([-0.0074, -0.1700,  0.0445,  0.1009, -0.0254], device='cuda:0',
       grad_fn=<SelectBackward0>)

Maximally activating dataset example: The cat was blue


##Fancier Examples

Looking for heads that mostly attend to the previous token


In [ ]:
long_text = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.'
print('Long text:', long_text)
# We first cache attention patterns
attn_cache = {}
def cache_attn(attn, hook):
    attn_cache[hook.name]=attn
logits = model.run_with_hooks(long_text, fwd_hooks=[(filter_hook_attn, cache_attn)])

# We then go through the cache and find the average attention paid to previous tokens
prev_token_scores = np.zeros((model.cfg['n_layers'], model.cfg['n_heads']))
for layer in range(model.cfg['n_layers']):
    for head in range(model.cfg['n_heads']):
        attn = attn_cache[f"blocks.{layer}.attn.hook_attn"][0, head]
        prev_token_scores[layer, head]=attn.diag(-1).mean().item()

px.imshow(prev_token_scores, 
          x=[f'Head {hi}' for hi in range(model.cfg['n_heads'])], 
          y=[f'Layer {i}' for i in range(model.cfg['n_layers'])], 
          title='Prev Token Scores', 
          color_continuous_scale='Blues')

Long text: Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.


[ROME style](https://rome.baulab.info/) patching for causal tracing - we have two runs with two different prompts and different answers, eg "Steve Jobs founded" -> " Apple" and "Bill Gates founded" -> " Microsoft". We patch parts of the layer outputs or residual stream from specific tokens and positions and see which patches significantly shift the answer from " Apple" to " Microsoft"

In [ ]:
prompt_1 = 'Bill Gates founded'
response_1 = ' Microsoft'
logit_index_1 = model.to_tokens(response_1)[0][-1]
(show_tokens(prompt_1))
prompt_2 = 'Steve Jobs founded'
response_2 = ' Apple'
logit_index_2 = model.to_tokens(response_2)[0][-1]
show_tokens(prompt_2)

model.reset_hooks()
uncorrupted_cache = {}
model.cache_all(uncorrupted_cache)
logits_1 = model(prompt_1)
model.reset_hooks()

uncorrupted_logits = model(prompt_2)
uncorrupted_log_probs = F.log_softmax(uncorrupted_logits, dim=-1)
print('Uncorrupted log prob for', response_1, uncorrupted_log_probs[0, -1, logit_index_1].item())
print('Uncorrupted log prob for', response_2, uncorrupted_log_probs[0, -1, logit_index_2].item())

# Patch the residual stream from the Bill Gates run to the Steve Jobs run
# at the Jobs/Gates token, at the start of layer 7
layer = 7
position = 1

def patch_resid_pre(resid_pre, hook):
    uncorrupted_resid_pre = uncorrupted_cache[hook.name]
    # Move things on the Jobs/Gates token
    resid_pre[:, position] = uncorrupted_resid_pre[:, position]
    return resid_pre

corrupted_logits = model.run_with_hooks(prompt_2, 
                    fwd_hooks=[(f'blocks.{layer}.hook_resid_pre', patch_resid_pre)])
corrupted_log_probs = F.log_softmax(corrupted_logits, dim=-1)
print('Corrupted (Residual) log prob for', response_1, corrupted_log_probs[0, -1, logit_index_1].item())
print('Corrupted (Residual) log prob for', response_2, corrupted_log_probs[0, -1, logit_index_2].item())

Bill| Gates| founded
Steve| Jobs| founded
Uncorrupted log prob for  Microsoft -2.890321731567383
Uncorrupted log prob for  Apple -0.557738184928894
Corrupted (Residual) log prob for  Microsoft -0.5379205346107483
Corrupted (Residual) log prob for  Apple -4.753251552581787


We can also patch the outputs of MLP layers 0 to 7 on the Gates/Jobs token - this time, rather than giving a hook name, we give a Boolean function that filters for the names of those hooks.

In [ ]:
layer_start = 0
layer_end = 7

def patch_mlp_post(mlp_post, hook):
    return uncorrupted_cache[hook.name]

def filter_middle_mlps(name):
    split_name = name.split('.')
    if split_name[-1]=='hook_post':
        layer = int(split_name[1])
        return (layer_start<=layer<layer_end)
    return False

corrupted_logits = model.run_with_hooks(prompt_2, 
                    fwd_hooks=[(filter_middle_mlps, patch_mlp_post)])
corrupted_log_probs = F.log_softmax(corrupted_logits, dim=-1)
print('Corrupted (MLP) log prob for', response_1, corrupted_log_probs[0, -1, logit_index_1].item())
print('Corrupted (MLP) log prob for', response_2, corrupted_log_probs[0, -1, logit_index_2].item())

Corrupted (MLP) log prob for  Microsoft -1.2620240449905396
Corrupted (MLP) log prob for  Apple -3.783123016357422


Looking for [induction heads](https://transformer-circuits.pub/2022/in-context-learning-and-induction-heads/index.html), by feeding in a random sequence of tokens repeated twice and looking for heads that attend from a second copy of a token to the token just after the first copy.

In [ ]:
seq_len = 100
rand_tokens = torch.randint(1000, 10000, (4, seq_len))
rand_tokens_repeat = einops.repeat(rand_tokens, 'batch pos -> batch (2 pos)')
if torch.cuda.is_available():
    rand_tokens_repeat = rand_tokens_repeat.cuda()

induction_scores_array = np.zeros((model.cfg['n_layers'], model.cfg['n_heads']))
def calc_induction_score(attn_pattern, hook):
    # Pattern has shape [batch, index, query_pos, key_pos]
    induction_stripe = attn_pattern.diagonal(1-seq_len, dim1=-2, dim2=-1)
    induction_scores = einops.reduce(induction_stripe, 'batch index pos -> index', 'mean')
    # Store the scores in a common array
    induction_scores_array[hook.layer()] = induction_scores.detach().cpu().numpy()
    
def filter_attn_hooks(hook_name):
    split_name = hook_name.split('.')
    return split_name[-1]=='hook_attn'

induction_logits = model.run_with_hooks(rand_tokens_repeat, fwd_hooks=[(filter_attn_hooks, calc_induction_score)])
px.imshow(induction_scores_array, labels={'y':'Layer', 'x':'Head'}, color_continuous_scale='Blues')

**Validation:** We can ablate the top few heads by this metric, and show that performance goes down substantially

In [ ]:
induction_logits = model(rand_tokens_repeat)
induction_log_probs = F.log_softmax(induction_logits, dim=-1)
induction_pred_log_probs = torch.gather(induction_log_probs[:, :-1], -1, rand_tokens_repeat[:, 1:, None])[..., 0]
print('Original loss on repeated sequence:', induction_pred_log_probs[:, seq_len:].mean())

# Mask out the heads with a high induction score
attn_head_mask = induction_scores_array>0.8

def prune_attn_heads(value, hook):
    # Value has shape [batch, pos, index, d_head]
    mask = attn_head_mask[hook.layer()]
    value[:, :, mask] = 0.
    return value

def filter_value_hooks(name):
    return name.split('.')[-1]=='hook_v'

ablated_logits = model.run_with_hooks(rand_tokens_repeat, fwd_hooks=[(filter_value_hooks, prune_attn_heads)])
ablated_log_probs = F.log_softmax(ablated_logits, dim=-1)
ablated_pred_log_probs = torch.gather(ablated_log_probs[:, :-1], -1, rand_tokens_repeat[:, 1:, None])[..., 0]
print('Loss on repeated sequence without induction heads:', ablated_pred_log_probs[:, seq_len:].mean())

px.imshow(attn_head_mask, labels={'y':'Layer', 'x':'Head'}, color_continuous_scale='Blues', title='Mask').show()

Original loss on repeated sequence: tensor(-0.1399, device='cuda:0', grad_fn=<MeanBackward0>)
Loss on repeated sequence without induction heads: tensor(-6.4074, device='cuda:0', grad_fn=<MeanBackward0>)


## Ablation experiments

We provide a wrapper to facilitate ablations experiment. 

An `EasyAblation` object is the combinaison of:
* The `EasyTransformer` model to be ablated
* An `AblationConfig` object that store all the parameters of the ablation 
* An `AblationMetric` object that define how we will mesure the effect of the ablation. This can be the loss on a given dataset or the attention score of a precise head.

Here, we defined a metric function that takes in inputs the model and the dataset and output a tensor. You can use `model.run_with_hook()` in the metric function, by you *have* to use the option `reset_hooks_start=False`, else the ablation hooks will be ignored. 

You can specify the `target_module` from "mlp", "attn_layer", "attn_head". 
If you chose "attn_head" you can define wich part of the head computation to ablate ("z", "q", "v", "k", "attn", "attn_scores") 

The supported ablation types are `mean`, `zero`, `neg` and `custom`. For mean ablations, you can specify a `mean_dataset` in the config that can be different to the one used in the metric. 

The `verbose` option prints all the experiment parameters before running the ablations.

In [ ]:
def induction_loss(model, dataset):
    induction_logits = model(dataset)
    induction_log_probs = F.log_softmax(induction_logits, dim=-1)
    induction_pred_log_probs = torch.gather(induction_log_probs[:, :-1], -1, rand_tokens_repeat[:, 1:, None])[..., 0]
    return induction_pred_log_probs[:, seq_len:].mean()

metric = AblationMetric(metric=induction_loss, dataset=rand_tokens_repeat, relative_metric=True)
config = AblationConfig(abl_type="zero", target_module="attn_head",head_circuit="z",  cache_means=True, verbose=True)
abl = EasyAblation(model, config, metric)
result = abl.run_ablation()

px.imshow(result, labels={'y':'Layer', 'x':'Head'}, color_continuous_scale='Blues', title='Induction Score Variation after Ablation').show()

Ablation config:
* abl_type: zero
* target_module: attn_head
* head_circuit: z
* layers: all
* heads: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
* mean_dataset: None
* dataset: tensor([9551, 4110, 7365, 1723, 5162, 6308, 4121, 8784, 9156, 5107],
       device='cuda:0') ... 
tensor([7952, 9394, 4904, 1543, 9594, 5548, 6921, 3332, 7552, 4954],
       device='cuda:0') ... 
tensor([9755, 8249, 7976, 5568, 3455, 3892, 1739, 8139, 9768, 2955],
       device='cuda:0') ... 
* verbose: True
* cache_means: True
* compute_means: False
* custom_abl_fn: None
* tokens_pos: all
* beg_layer: 0
* end_layer: 12

Ablation config:
* abl_type: zero
* target_module: attn_head
* head_circuit: z
* layers: all
* heads: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
* mean_dataset: None
* dataset: tensor([2032, 7746, 4167, 6423, 2320, 3057, 7675, 3826, 8123, 9653],
       device='cuda:0') ... 
tensor([7952, 9394, 4904, 1543, 9594, 5548, 6921, 3332, 7552, 4954],
       device='cuda:0') ... 
tensor([9755, 8249, 7976, 5568, 3

100%|██████████| 12/12 [00:12<00:00,  1.05s/it]


You can also use the `EasyAblation` object to generate ablations hook, without using it to run ablations. This is useful when you want to ablate several heads at once. The ablations hooks generated will respect the configuration you used to generate the `EasyAblation` object.

Here we can reproduce the revious results where we ablate the induction heads by replacing their activation by zero. You can notice that the combined effect of the ablations of the 5 heads is much greater than the sum of their individual effect.

In [ ]:
model.reset_hooks()
for (l,h) in [(5,1), (5,5), (6,9), (7,2), (7,10)]:
    hook_name, hook = abl.get_head_ablation_hook(l,h)
    model.add_hook(hook_name, hook)
print(f"Loss on the repeated random token after zero-ablations of the induction heads {induction_loss(model, rand_tokens_repeat)}")

Loss on the repeated random token after zero-ablations of the induction heads -6.407426357269287


However, replacing the activation by zero is quite a weird thing to do as this could be really far from the baseline activation depending on the head. To fix this, we can instead replace the activation by its mean on the dataset. All the effect related to a particular sample will be washed out but the global contribution will be kept. The drop is still significant but not as much as zero ablation.

In [ ]:
mean_abl_config = AblationConfig(abl_type="mean", target_module="attn_head",head_circuit="z", cache_means=True)
mean_abl = EasyAblation(model, mean_abl_config, metric)
model.reset_hooks()
for (l,h) in [(5,1), (5,5), (6,9), (7,2), (7,10)]:
    hook_name, hook = mean_abl.get_head_ablation_hook(l,h)
    model.add_hook(hook_name, hook)
    
print(f"Loss on the repeated random token after mean-ablations of the induction heads {induction_loss(model, rand_tokens_repeat)}")

Loss on the repeated random token after mean-ablations of the induction heads -1.8049614429473877


### Custom ablation function

When using `abl_type="custom"` you can specify an arbitrary function `custom_abl_fn`. It has to take as input the normal output of the module, its mean activation, the hook object and output a tensor of the same shape as the normal output.

The mean as the same shape as the normal output, its constant along the batch_size dimension. 

If `target_module="attn_head"` the output is for a given head, its shape would be `(batch, seq_len, head_dim)` except for attention score and attention pattern : `(batch,seq_len, seq_len)`.

In the example below, we take the symetric of the activation with respect to its mean. It has for effect to reverse the contribution of the head without going too far in out of distribution activation space (as naively flipping the sign would do).

In [ ]:

def sym_mean(z, mean, hook): 
    return mean-(z-mean)

metric = AblationMetric(induction_loss, rand_tokens_repeat, relative_metric=True)
config = AblationConfig(abl_type="custom",custom_abl_fn=sym_mean, target_module="attn_head",head_circuit="v",  cache_means=True, verbose=False)
abl = EasyAblation(model, config, metric)
result = abl.run_ablation()
px.imshow(result, labels={'y':'Layer', 'x':'Head'}, color_continuous_scale='Blues', title='Induction Score Variation after Custom Ablation').show()

100%|██████████| 12/12 [00:12<00:00,  1.07s/it]
